In [1]:
import pandas as pd
import xgboost as xgb

In [2]:
%cd /Users/vladimirkalajcidi/avito_mlcup

/Users/vladimirkalajcidi/avito_mlcup


In [3]:
train = pd.read_parquet("data/train.parquet")

test = pd.read_parquet("data/test.parquet")
users = test['user_id'].unique()
items = test['adv_campaign_id'].unique()

train = train[train['user_id'].isin(users)][train['adv_campaign_id'].isin(items)]
del test
del users
del items

/var/folders/mx/gtrgssgs20v5hz0g3m4nnm7w0000gn/T/ipykernel_63878/3310709516.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train = train[train['user_id'].isin(users)][train['adv_campaign_id'].isin(items)]


In [4]:
campaigns = pd.read_parquet("data/campaigns_meta.parquet").drop(columns=['start_date', 'end_date', 'location_id'])
train = pd.merge(train, campaigns, on='adv_campaign_id', how='left')
del campaigns

In [5]:
train = train.drop(columns=['event_date'])

In [6]:
new_mapping = {
    True: 1,
    False: 0
}

# Use replace to update the interaction column
train['is_main'] = train['is_main'].replace(new_mapping)

/var/folders/mx/gtrgssgs20v5hz0g3m4nnm7w0000gn/T/ipykernel_63878/1225067986.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['is_main'] = train['is_main'].replace(new_mapping)


In [7]:
train.head()

,user_id,adv_campaign_id,platform_id,adv_creative_id,banner_code,is_main,target,goal_cost,goal_budget,logcat_id
0,1555200,2226,2,1764,8,1,0,2.478521,16573.003850,40
1,2589012,3113,2,4223,8,1,0,6.254672,5402.003810,7
2,3192789,3366,3,2360,6,1,0,3.719954,11673.174111,18
3,29114,1996,2,413,8,1,0,2.586311,3695.440619,40
4,2243477,3095,2,595,8,1,0,3.226281,4010.265212,40


In [8]:
train = train.drop(columns=['user_id', 'adv_campaign_id'])

In [9]:
train.head()

,platform_id,adv_creative_id,banner_code,is_main,target,goal_cost,goal_budget,logcat_id
0,2,1764,8,1,0,2.478521,16573.003850,40
1,2,4223,8,1,0,6.254672,5402.003810,7
2,3,2360,6,1,0,3.719954,11673.174111,18
3,2,413,8,1,0,2.586311,3695.440619,40
4,2,595,8,1,0,3.226281,4010.265212,40


In [22]:
params = {
    'tree_method': 'exact',
    'objective': 'binary:logistic'
}
num_boost_round = 500
class_weight = len(train[train['target'] == 0]) / len(train[train['target'] == 1])

clf = xgb.XGBClassifier(n_estimators=num_boost_round, scale_pos_weight=class_weight, **params)
clf.fit(train.drop(columns=['target']), train['target'], 
        verbose=10)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [23]:
test = pd.read_parquet("data/test.parquet")
campaigns = pd.read_parquet("data/campaigns_meta.parquet").drop(columns=['start_date', 'end_date', 'location_id'])

test = pd.merge(test, campaigns, on='adv_campaign_id', how='left')
del campaigns

test = test.drop(columns=['event_date'])

# Use replace to update the interaction column
test['is_main'] = test['is_main'].replace(new_mapping)

test = test.drop(columns=['user_id', 'adv_campaign_id'])

/var/folders/mx/gtrgssgs20v5hz0g3m4nnm7w0000gn/T/ipykernel_63878/3709893115.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['is_main'] = test['is_main'].replace(new_mapping)


In [24]:
result = clf.predict_proba(test)
result = [i[1] for i in result]

test = pd.read_parquet("data/test.parquet")
test['predict'] = result

In [25]:
test[['user_id', 'adv_campaign_id', 'predict']].to_csv('subs/xgb_500_est_onlyontestusersanditems.csv', index=False)